In [1]:
import numpy as np

from nanotag.files import ImageCollectionBrowser, walk_dir
from nanotag.canvas import Canvas
from nanotag.artists import ImageArtist, ScatterArtist
from nanotag.utils import link
from nanotag.image import GaussianFilterSlider
from nanotag.tools import BoxZoomTool, PanZoomTool

import ipywidgets as widgets
from traitlets import directional_link

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [2]:
#files = walk_dir('D:\\ucloud\\Shared\\STEM-R18\\ScanMap\\Hole-65\\map_2020_09_29_10_34', '.tif')
#files = walk_dir('D:\\data\\MoS2_example_series', '.tif')
files = walk_dir('D:\\data\\julia', '.tif')

metadata = {files[0]:'metadata.json'}

In [3]:


from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox
from ipyevents import Event 
from IPython.display import display



In [31]:
from traitlets import HasTraits, Any, Float, Int
from bqplot_image_gl.interacts import MouseInteraction
from scipy.spatial import KDTree




class EditPointTool(HasTraits):
    scatter_artist = Any()
    tolerance = Float(50)
    _selected = Int(None, allow_none=True)

    def activate(self, canvas):
        interaction = MouseInteraction(x_scale=canvas.x_scale, y_scale=canvas.y_scale)        
        
        def on_msg(_, change, __):
            position = [change['domain']['y'], change['domain']['x']]
            
            if change['event'] in ('click',):
            
                if change['ctrlKey']:
                    distance, index = KDTree(self.scatter_artist.points).query(position, 1)
                    
                    if distance > self.tolerance:
                        return
                    
                    self.scatter_artist.points = np.delete(self.scatter_artist.points, index, axis=0)
                    
                else:
                    self.scatter_artist.points = np.vstack((self.scatter_artist.points, [position]))
            
            
#             if change['event'] in ('dragmove',):
#                 distance, index = KDTree(self.scatter_artist.points).query(position, 1)
#                 self._selected = index.item()
                
#                 if self._selected is None:
#                     return
                
#                 if len(self.scatter_artist.points) == 0:
#                     return
                
#                 points = self.scatter_artist.points
#                 points[self._selected][0] = position[0]
#                 points[self._selected][1] = position[1]
                
#                 self.scatter_artist._mark.y = points[:,0]
#                 self.scatter_artist._mark.x = points[:,1]
                
#                 print('aaa')
            
#             if change['event'] in ('dragend',):
#                 self._selected = None
        
        canvas.figure.interaction = interaction
        interaction.events = []
        #interaction.on_msg(on_msg)

    def deactivate(self, canvas):
        #self.scatter_artist._mark.enable_move = False
        canvas.figure.interaction = None
        

In [18]:
browser = ImageCollectionBrowser(paths = files, binning=1, metadata_paths=metadata)

canvas = Canvas()
canvas.x_label = 'x [pixels]'
canvas.y_label = 'y [pixels]'

image_artist = ImageArtist()
point_artist = ScatterArtist()

canvas.artists = {'points':point_artist, 'image':image_artist,}
canvas.tools = {'Pan': PanZoomTool(), 'Zoom': BoxZoomTool()}#, 'Edit':EditPointTool(scatter_artist=point_artist)}

def transform_link():
    def _transform_link(x):
        try:
            return np.array(x['points'])
        except KeyError:
            return np.zeros((0,2), dtype=np.float64)
    
    return _transform_link

directional_link((browser, 'frame_metadata'), (point_artist, 'points'), transform= transform_link())

gaussian_filter = GaussianFilterSlider()

link((browser, 'image'), (gaussian_filter, 'image_in'), check_broken=False)
link((gaussian_filter, 'image_out'), (image_artist, 'image'), check_broken=False)

browser._observe_frame_index()

app = widgets.VBox([widgets.HBox([canvas.widget, widgets.VBox([browser.widget, gaussian_filter])]), canvas.toolbar])

app

In [22]:
point_artist._mark.enable_move = True
#point_artist._mark.interactions  = {'click':'add'}

In [20]:
l = Label('Click or type on me!')
l.layout.border = '2px solid red'

h = HTML('Event info')
d = Event(source=app, watched_events=['keydown'])

def handle_event(event):
    if event['key'] == 'p':
        canvas.tool = 'Pan'
    
    if event['key'] == 'z':
        canvas.tool = 'Zoom'
    
    if event['key'] == 'r':
        canvas.reset()
    
    if event['key'] == 'ArrowRight':
        browser.next_frame()
    
    if event['key'] == 'ArrowLeft':
        browser.previous_frame()
    
    
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    h.value = content

d.on_dom_event(handle_event)
                            
display(l, h)

Label(value='Click or type on me!', layout=Layout(border='2px solid red'))

HTML(value='Event info')

In [306]:
point_artist._mark.enable_move = True

In [205]:
point_artist.points = np.vstack((point_artist.points, [[30,30]]))

In [170]:
from bqplot import Interaction
from bqplot_image_gl.interacts import MouseInteraction, keyboard_events, mouse_events

In [176]:
interaction = MouseInteraction(x_scale=canvas.x_scale, y_scale=canvas.y_scale, #next=Interaction(),
                              events = keyboard_events)

canvas.figure.interaction = interaction
#interaction.events = ['click']

def on_mouse_msg(*args):
    print('aa')

interaction.on_msg(on_mouse_msg)

In [175]:
point_artist._mark.interactions = {'click': 'add'}
point_artist._mark.enable_move = True

In [65]:
point_artist._mark

Scatter(colors=['red'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(allow_padding=False), 'y': LinearScale(allow_padding=False)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9})

In [61]:
point_artist._mark.x[20] = 20

ValueError: assignment destination is read-only

In [16]:
len(point_artist.points)

20

In [61]:
canvas.x_scale

LinearScale(allow_padding=False, max=511.5, min=-0.5)

In [49]:
point_artist.limits

ValueError: zero-size array to reduction operation minimum which has no identity

In [46]:
canvas.toolbar

In [38]:
point_artist._mark.x = np.array([10.,15.])
point_artist._mark.y = np.array([10.,15.])

In [34]:
canvas.figure.marks[0].y

array([-5.000e-01,  5.115e+02])

In [22]:
point_artist._mark.colors = ['blue']

In [16]:
point_artist.x

array([], dtype=float64)

In [9]:
browser.frame_metadata

{'points': [[249.34733147332986, -37.53690065652258],
  [544.1574445081301, -34.78811093100858],
  [40.2181405983585, -34.85572044762844],
  [337.8337595902933, -29.48465964484098],
  [126.21107043130766, -22.91208559761767],
  [385.1810410368295, -21.478162782321824],
  [171.61936144237427, -15.96780863669764],
  [-32.17645585163819, -8.3755146937873],
  [475.9096117250294, -5.3357538748753015],
  [262.7907813686932, 1.890192542548087],
  [521.2567453403831, 2.7341668102083445],
  [52.20820407574241, 5.797074833114087],
  [308.33833894031153, 8.036876028244913],
  [98.81751728852878, 13.431449141558405],
  [400.1148398061421, 22.389122700542604],
  [187.75389417184925, 29.49936306591128],
  [445.43642496792495, 31.901764724056115],
  [-21.34950604515379, 34.85481138650903],
  [233.26417289157223, 37.81157639444403],
  [24.128114021443192, 42.89666782819559],
  [537.2162683557178, 46.09577493021169],
  [324.28923460098923, 52.527417223458265],
  [114.61460689580645, 57.14667300198967],

In [25]:
browser.image

array([[0.00216649, 0.00226497, 0.00190389, ..., 0.00157563, 0.00196954,
        0.00203519],
       [0.00226497, 0.00249475, 0.00239627, ..., 0.00210084, 0.00236345,
        0.00200236],
       [0.00216649, 0.0015428 , 0.00298713, ..., 0.0024291 , 0.00180541,
        0.00236345],
       ...,
       [0.00183824, 0.00177258, 0.00206801, ..., 0.00167411, 0.00311843,
        0.00200236],
       [0.00236345, 0.00164128, 0.0014115 , ..., 0.00196954, 0.00216649,
        0.00216649],
       [0.00226497, 0.00262605, 0.00236345, ..., 0.00170693, 0.00160846,
        0.00196954]], dtype=float32)

In [11]:
canvas.figure.axes[0].label = 'sss'